In [83]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

import requests
from urllib.parse import urlencode

### api yandex download

In [84]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

url_dict = {
        'task_one'  : {'file_name': 'olist_orders_dataset.csv'     , 'public_key': 'https://disk.yandex.ru/d/0zWmh1bYP6REbw'},
        'task_two'  : {'file_name': 'olist_order_items_dataset.csv', 'public_key': 'https://disk.yandex.ru/d/xCsQ0FiC-Ue4Lg'},
        'task_three': {'file_name': 'olist_customers_dataset.csv'  , 'public_key': 'https://disk.yandex.ru/d/QXrVDUoQvFhIcg'}
        }

In [85]:
def download_files(url_dictionary, dataset):
    # ссылка для загрузки
    final_url = base_url + urlencode(dict(public_key=url_dictionary[dataset]['public_key']))
    response = requests.get(final_url)
    download_url = response.json()['href']
    
    # сохранение файла в датафрейм
    df = pd.read_csv(download_url)
    return df

In [103]:
orders_data = download_files(url_dict, 'task_one')
items_data = download_files(url_dict, 'task_two')
customers_data = download_files(url_dict, 'task_three')

### Предварительный анализ


In [87]:
orders_data.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [88]:
orders_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [89]:
items_data.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [90]:
items_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [91]:
customers_data.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [92]:
customers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


### Задание номер 1

In [93]:
one_purchase = pd.merge(orders_data, customers_data, on='customer_id')
one_purchase.shape[0], orders_data.shape[0], customers_data.shape[0]

(99441, 99441, 99441)

In [94]:
one_purchase.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
dtype: int64

### Видим пустые значения, считаем их как не совсем корректными, так же фильтруем группы canceled, unavailable.

In [95]:
clients = one_purchase \
    .query("order_status != 'unavailable' and order_status != 'canceled'") \
    .dropna(subset=['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']) \
    .groupby('customer_unique_id', as_index=False) \
    .agg({'customer_id': 'count'}) \
    .sort_values('customer_id', ascending=False)
print("Пользователей совершившие всего одну покупку -", clients.query('customer_id == 1').shape[0])

Пользователей совершившие всего одну покупку - 90536


### Задание номер 2

### Считаем что не доставленный заказ это тот заказ, который оплачен, но не имеет статуса доставлен

In [96]:
not_a_sucess = orders_data.query("order_approved_at.notna() and order_status != 'delivered' and order_status != ['canceled', 'unavailable']")
not_a_sucess = not_a_sucess.astype({'order_approved_at' : 'datetime64[M]'}) 

In [97]:
not_a_sucess = not_a_sucess.groupby(['order_status', 'order_approved_at'], as_index=False) \
    .agg({'order_id': 'count'})

In [98]:
mean_not_a_sucess = not_a_sucess \
    .groupby('order_approved_at', as_index=False) \
    .agg({'order_id': 'mean'}) \
    .rename(columns={'order_id': 'count_id'})
print("В среднем в месяц не доставляется -", mean_not_a_sucess.count_id.mean().round(), "заказов")

В среднем в месяц не доставляется - 27.0 заказов


### Задание номер 3

### День покупки считаем день подтверждения оплаты

In [104]:
orders_data_approved = orders_data \
    .dropna(subset=['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']) \
    .query("order_status != 'unavailable' and order_status != 'canceled'")

In [108]:
sell_items = items_data
sell_items["order_approved_at"] = sell_items["order_id"] \
    .map(orders_data_approved.set_index("order_id")["order_approved_at"])

sell_items["order_approved_at"] = pd.to_datetime(sell_items["order_approved_at"])
sell_items["day_approve"] = items_sell["order_approved_at"].dt.day_name()

In [109]:
sell_items = sell_items \
    .groupby(['product_id', 'day_of_approve'])\
    .agg({'day_approve': 'count'})\
    .rename(columns ={'day_approve': 'count_days_approve'}).reset_index()

In [114]:
sell_items_days_max = sell_items.loc[sell_items.groupby('product_id')['count_days_approve'].idxmax()]
sell_items_days_max

,product_id,day_of_approve,count_days_approve
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday,1
1,00088930e925c41fd95ebfe695fd2655,Tuesday,1
2,0009406fd7479715e4bef61dd91f2462,Friday,1
3,000b8f95fcb9e0096488278317764d19,Friday,1
5,000d9be29b5207b54e86aa1b1ac54872,Tuesday,1
...,...,...,...
59822,fff6177642830a9a94a0f2cba5e476d1,Monday,1
59824,fff81cc3158d2725c0655ab9ba0f712c,Monday,1
59825,fff9553ac224cec9d15d49f5a263411f,Saturday,1
59828,fffdb2d0ec8d6a61f0a0a0db3f25b441,Tuesday,3


### На выходе имеем таблицу в котором подсчитан день максимального кол-во покупок

### Задание номер 4 p.s(изменена логика отнесения заказа к покупке)

In [115]:
# мерджим датафреймы
buy_total_week = orders_data_approved.merge(customers_data, on = 'customer_id').sort_values('order_approved_at').query('order_approved_at.notna()')
buy_total_week.shape[0], orders_data.shape[0], customers_data.shape[0]

(96455, 99441, 99441)

In [116]:
# проверка на NaN
buy_total_week.isna().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
dtype: int64

In [117]:
# присваеваем новые колонки для ф-ии
buy_total_week['order_approved_at'] = pd.to_datetime(buy_total_week['order_approved_at'])
buy_total_week['month'] = buy_total_week['order_approved_at'].dt.month
buy_total_week['year'] = buy_total_week['order_approved_at'].dt.year

In [118]:
def sum_days_of_month(month, year):
    days_in_month = [31, 28 + int((year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))), 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    if 1 <= month <= 12:
        return days_in_month[month - 1]
    else:
        return None

In [119]:
# применяем ф-ию
buy_total_week['number_weeks'] = buy_total_week[['month', 'year']].apply(lambda x: sum_days_of_month(*x), axis = 1) / 7

In [120]:
average_sales_by_week = buy_total_week \
    .groupby(['customer_unique_id', 'year', 'month', 'number_weeks'], as_index = False) \
    .agg({'order_id' : 'count'})

In [121]:
# кол-во покупок в неделю
average_sales_by_week['sales_week'] = average_sales_by_week.order_id / average_sales_by_week.number_weeks
average_sales_by_week

,customer_unique_id,year,month,number_weeks,order_id,sales_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018,5,4.428571,1,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018,5,4.428571,1,0.225806
2,0000f46a3911fa3c0805444483337064,2017,3,4.428571,1,0.225806
3,0000f6ccb0745a6a4b88665a16c9f078,2017,10,4.428571,1,0.225806
4,0004aac84e0df4da2b147fca70cf8255,2017,11,4.285714,1,0.233333
...,...,...,...,...,...,...
95162,fffcf5a5ff07b0908bd4e2dbc735a684,2017,6,4.285714,1,0.233333
95163,fffea47cd6d3cc0a88bd621562a9d061,2017,12,4.428571,1,0.225806
95164,ffff371b4d645b6ecea244b27531430a,2017,2,4.000000,1,0.250000
95165,ffff5962728ec6157033ef9805bacc48,2018,5,4.428571,1,0.225806


In [122]:
# среднее кол во покупок в неделю
average_sales_by_week.sales_week.mean()

0.23327160822328055

### Задание номер 5 p.s(изменена логика отнесения заказа к покупке)

In [124]:
# единсственный полный год 2017-й. замерджю дата фреймы и отфильтрую их по датам, так же отредактирую дата фрейм с помощью
# метода .pivot    -

In [130]:
retention = customers_data.merge(orders_data_approved, on='customer_id').fillna(0)
retention['order_purchase_timestamp'] = retention['order_purchase_timestamp'].astype('datetime64[M]')

In [131]:
retention_month = retention.groupby('customer_unique_id')['order_purchase_timestamp'].min().reset_index()
retention_month = retention_month.rename(columns={'order_purchase_timestamp': 'first_month'})

In [132]:
kogorta = retention.merge(retention_month,on='customer_unique_id').query('first_month >= "2017-01-01" and first_month <= "2017-12-01" and order_purchase_timestamp <= "2018-03-01"')
kogorta['kogorta_month'] = kogorta['order_purchase_timestamp'].dt.to_period('M').astype(int)-kogorta['first_month'].dt.to_period('M').astype(int)

In [133]:
kogorta = kogorta.pivot_table(index='first_month',columns='kogorta_month',values='customer_unique_id',aggfunc='nunique')
kogorta

kogorta_month,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
first_month,,,,,,,,,,,,,,,
2017-01-01,715.0,2.0,2.0,1.0,3.0,1.0,3.0,1.0,1.0,NaN,3.0,1.0,5.0,3.0,1.0
2017-02-01,1616.0,3.0,5.0,2.0,7.0,2.0,4.0,3.0,1.0,3.0,2.0,5.0,2.0,3.0,NaN
2017-03-01,2503.0,11.0,9.0,10.0,9.0,4.0,4.0,8.0,8.0,2.0,9.0,3.0,5.0,NaN,NaN
2017-04-01,2256.0,14.0,5.0,4.0,6.0,6.0,8.0,7.0,7.0,4.0,6.0,2.0,NaN,NaN,NaN
2017-05-01,3450.0,16.0,16.0,10.0,10.0,11.0,14.0,5.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN
2017-06-01,3037.0,15.0,12.0,13.0,9.0,12.0,11.0,7.0,4.0,6.0,NaN,NaN,NaN,NaN,NaN
2017-07-01,3752.0,20.0,13.0,9.0,11.0,8.0,12.0,4.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-01,4057.0,28.0,14.0,11.0,14.0,21.0,12.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-09-01,4003.0,28.0,22.0,11.0,18.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
kogorta[3] / kogorta[0] * 100

first_month
2017-01-01    0.139860
2017-02-01    0.123762
2017-03-01    0.399521
2017-04-01    0.177305
2017-05-01    0.289855
2017-06-01    0.428054
2017-07-01    0.239872
2017-08-01    0.271136
2017-09-01    0.274794
2017-10-01    0.092400
2017-11-01    0.169996
2017-12-01    0.337205
dtype: float64